<a href="https://colab.research.google.com/github/taryaksama/data-science/blob/master/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
#import packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Clone a GitHub repository
!git clone https://github.com/taryaksama/data-science/
%cd data-science

In [ ]:
pathfile = 'Task1_files/'
file1 = '240614_177_10pc_frame_0_voronoi_areas_microns.csv'
file2 = '240614_177_10pc_surface_coverage_and_density.csv'

filename = pathfile+file1
df = pd.read_csv(filename)
df.columns = ['cellid', 'area', 'area_mean', 'area_norm']

In [ ]:
df

# 1. Histogram Area

In [ ]:
# get counts of histogram
bin_edges = list(range(100))
bin_edges_norm = list(np.linspace(0,1,11))

counts, _, _ = plt.hist(df['area'], bins=bin_edges, density=True)
counts_norm, _, _ = plt.hist(df['area_norm'], bins=bin_edges_norm, density=True)
plt.close()

In [ ]:
# display histogram
fig, axes = plt.subplots(1,2, figsize=(10, 5))
sns.despine(f)

## area
sns.histplot(df['area'], bins=bin_edges, stat='density', element='poly', alpha=0.5, ax=axes[0])
axes[0].set_title('Area')

## normalized area
sns.histplot(df['area_norm'], bins=bin_edges_norm, stat='density', element='poly', alpha=0.5, ax=axes[1])
axes[1].set_title('Normalized Area')

# 2. Surface Coverage